<a href="https://colab.research.google.com/github/Patortiz26/Laboratorio-de-Programaci-n/blob/main/Laboratorio8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo:

- Patricio Ortiz


### **Link de repositorio de GitHub:** `https://github.com/Patortiz26/Laboratorio-de-Programaci-n/`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Exploración del dataset y los resultados a través de visualizaciones interactivas usando `plotly`.
- Introducción a procesamiento del lenguaje natural.
- Clasificación en `scikit-learn`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 26/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar las ventajas que nos ofrece crear un pipeline.
- Obtener caracteristicas desde texto.
- Visualizar el funcionamiento de clasificadores.
- Realizar una GridSearch sobre un conjunto de clasificadores.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/My Drive/Colab Notebooks/MDS7202 - Laboratorio de programación/Lab 8/'
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_comics = pd.read_csv(path+'df_comics.csv')
df_comics_no_label = pd.read_csv(path+'comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [4]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

Unnamed: 0          name  ... has_agility has_super_strength
0           0       3-D Man  ...         0.0                1.0
1           2        A-Bomb  ...         1.0                1.0
2           3            Aa  ...         0.0                0.0
3           4    Aaron Cash  ...         0.0                0.0
4           5  Aayla Secura  ...         1.0                0.0

[5 rows x 82 columns]

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `bag of words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `bag of words`?...

`bag of words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `bag of words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `bag of words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [5]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [6]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [7]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [8]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [9]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

&  actor  beauti  best  citi  good  ...  new  rock  roll  teacher  world  york
0  1      0       0     0     0     1  ...    0     2     1        1      0     0
1  0      1       0     1     0     0  ...    0     1     0        0      1     0
2  0      0       1     0     1     0  ...    1     0     0        0      0     1

[3 rows x 13 columns]

Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [10]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

&  & roll  actor  actor world  ...  teacher rock  world  york  york beauti
0  1       1      0            0  ...             1      0     0            0
1  0       0      1            1  ...             0      1     0            0
2  0       0      0            0  ...             0      0     1            1

[3 rows x 25 columns]

De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [x] Obtener a traves de bag of words caracteristicas del resumen de historia de cada personaje.
- [x] Aplicar MinMaxScaler sobre los atributos de interes.

**Respuesta:**

In [11]:
bog = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )

numeric_features = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
numeric_transformer = MinMaxScaler()

preprocessor = ColumnTransformer(
    transformers = [
                    ('Texto', bog, 'history_text'),
                    ('Numeric', numeric_transformer, numeric_features),
    ])

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>


Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Finalmente entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

**To-DO:**
- [x] Realizar un pipeline con las caracteristicas solicitadas en 1.1 y aplicar un clasificador  `MultinomialNB()`.
- [x] Entrenar el pipeline y comentar los resultados.

**Respuesta:**

In [12]:
#Primero se separan los datos en train y test
features = df_comics.loc[:, ['history_text','intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']]
labels = df_comics.loc[:, "alignment"]

#Analizar balance/desbalance de las clases
print("Número de registros por clase:\n", labels.value_counts(),'\n')

#División en train y test
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.25, shuffle=True, stratify=labels, random_state=0
)

#Creamos pipeline
pipe = Pipeline(steps=[
                           ('Procesador', preprocessor),
                           ("Selection", SelectPercentile(f_classif, percentile=70)),
                           ('Clasificador', MultinomialNB())
])

#Ajustamos pipeline
pipe.fit(X_train, y_train)

#Predicción
y_pred = pipe.predict(X_test)

#Resultados
print("Matriz de confusión\n", confusion_matrix(y_pred, y_test),'\n')
print("Reporte de clasificación\n", classification_report(y_test, y_pred))

Número de registros por clase:
 Good       743
Bad        429
Neutral    113
Name: alignment, dtype: int64 

Matriz de confusión
 [[ 12   3   1]
 [ 96 183  26]
 [  0   0   1]] 

Reporte de clasificación
               precision    recall  f1-score   support

         Bad       0.75      0.11      0.19       108
        Good       0.60      0.98      0.75       186
     Neutral       1.00      0.04      0.07        28

    accuracy                           0.61       322
   macro avg       0.78      0.38      0.34       322
weighted avg       0.69      0.61      0.50       322



Se observa que hay un fuerte desbalance entre las clases, siendo "Good" la clase más frecuente y "Neutral" la clase con menor presencia en el dataset. Lo anterior genera que el modelo tienda a sobre estimar el número de etiquetas "Good" predichas. En consecuencia, analizar el accuracy del modelo nos induce un sesgo, y por tanto convendría analizar otra métrica. Teniendo eso en mente notamos que:

- Para la clase "Bad" y "Neutral": Tenemos un alto precision y un bajo recall, lo cual indica que el modelo no detecta la clase muy bien, pero cuando lo hace es altamente confiable.

- Para la clase "Good": Tenemos un alto recall y un precision menor, lo que significa que el modelo detecta bien la clase pero también incluye muestras de otras clases.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Dos clasificadores distintos en donde varie sus parámetros. Se le recomienda utilizar `LogisticRegression()` y `RandomForestClassifier()`.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. 
- Selección de las mejores columnas para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
        'clf__multi_class': ['liblinear']},
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [13]:
mb_pipe = Pipeline(steps=[
                           ('Procesador', preprocessor),
                           ("Selection", SelectPercentile(f_classif, percentile=70)),
                           ('Clasificador', MultinomialNB(alpha = 1, fit_prior= True))
])

random_pipe = Pipeline(steps=[
                           ('Procesador', preprocessor),
                           ("Selection", SelectPercentile(f_classif, percentile=70)),
                           ('Clasificador', RandomForestClassifier(n_estimators = 100, criterion = 'gini'))
])

logit_pipe = Pipeline(steps=[
                           ('Procesador', preprocessor),
                           ("Selection", SelectPercentile(f_classif, percentile=70)),
                           ('Clasificador', LogisticRegression(C = 1, solver = 'lbfgs'))
])

mb_param_grid = [{
    'Procesador__Texto__ngram_range' : [(1,1),(1,2),(1,3)],
    'Selection__percentile' : [20,40,60,80],
    'Clasificador__alpha' : [0.5, 1],
    'Clasificador__fit_prior' : [True, False]
}]

random_param_grid = [{
    'Procesador__Texto__ngram_range' : [(1,1),(1,2),(1,3)],
    'Selection__percentile' : [20,40,60,80],
    'Clasificador__n_estimators' : [80,100,120],
    'Clasificador__criterion' : ['gini','entropy']
}]

logit_param_grid = [{
    'Procesador__Texto__ngram_range' : [(1,1),(1,2),(1,3)],
    'Selection__percentile' : [20,40,60,80],
    'Clasificador__C' : [0.5, 1, 2],
    'Clasificador__solver' : ['lbfgs', 'saga']
}]

In [14]:
mb_hgs = HalvingGridSearchCV(mb_pipe, mb_param_grid, random_state = 0, n_jobs=-1).fit(X_train,y_train)

In [15]:
random_hgs = HalvingGridSearchCV(random_pipe, random_param_grid, random_state = 0, n_jobs = -1).fit(X_train,y_train)

In [16]:
logit_hgs = HalvingGridSearchCV(logit_pipe, logit_param_grid, random_state = 0, n_jobs = -1).fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [17]:
y_pred_mb = mb_hgs.predict(X_test)
y_pred_random = random_hgs.predict(X_test)
y_pred_logit = logit_hgs.predict(X_test)

In [18]:
#Resultados
print("Resultados para MB\n")
print("Matriz de confusión\n", confusion_matrix(y_pred_mb, y_test),'\n')
print("Reporte de clasificación\n", classification_report( y_pred_mb, y_test))

Resultados para MB

Matriz de confusión
 [[ 55  29  11]
 [ 49 153  15]
 [  4   4   2]] 

Reporte de clasificación
               precision    recall  f1-score   support

         Bad       0.51      0.58      0.54        95
        Good       0.82      0.71      0.76       217
     Neutral       0.07      0.20      0.11        10

    accuracy                           0.65       322
   macro avg       0.47      0.49      0.47       322
weighted avg       0.71      0.65      0.67       322



In [19]:
#Resultados
print("Resultados para Random Forest\n")
print("Matriz de confusión\n", confusion_matrix(y_pred_random, y_test),'\n')
print("Reporte de clasificación\n", classification_report( y_pred_random, y_test))

Resultados para Random Forest

Matriz de confusión
 [[ 24   6   4]
 [ 84 180  24]
 [  0   0   0]] 

Reporte de clasificación
               precision    recall  f1-score   support

         Bad       0.22      0.71      0.34        34
        Good       0.97      0.62      0.76       288
     Neutral       0.00      0.00      0.00         0

    accuracy                           0.63       322
   macro avg       0.40      0.44      0.37       322
weighted avg       0.89      0.63      0.71       322



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
#Resultados
print("Resultados para logit\n")
print("Matriz de confusión\n", confusion_matrix(y_pred_logit, y_test),'\n')
print("Reporte de clasificación\n", classification_report( y_pred_logit, y_test))

Resultados para logit

Matriz de confusión
 [[ 61  24   8]
 [ 44 158  17]
 [  3   4   3]] 

Reporte de clasificación
               precision    recall  f1-score   support

         Bad       0.56      0.66      0.61        93
        Good       0.85      0.72      0.78       219
     Neutral       0.11      0.30      0.16        10

    accuracy                           0.69       322
   macro avg       0.51      0.56      0.52       322
weighted avg       0.74      0.69      0.71       322



## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [21]:
df_comics_no_label = df_comics_no_label.drop_duplicates(subset = ['name'])
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text']) # eliminar ejemplos sin historia
df_comics_no_label = df_comics_no_label.reset_index(drop=True)
df_comics_no_label = df_comics_no_label.drop(columns = ['alignment'])

X = df_comics_no_label.loc[:, ['history_text','intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']]

#Se elige logit porque entregó los mejores resultados
pred = logit_hgs.predict(X)

df_comics_no_label['alignment'] = pred

In [22]:
lista = ['Vergil', 'Gorilla Girl' , 'Batcow']
df_comics_no_label[df_comics_no_label.name.isin(lista)].loc[:,['name','alignment']]

name alignment
14        Batcow      Good
37  Gorilla Girl      Good
71        Vergil      Good

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>